## Analytics on the cloud



-   Deploy computing resources on the cloud
-   But what if we want to perform operations within frameworks on the cloud?
-   We will focus on Google Earth Engine but other solutions exist



## Google Earth Engine



![img](images/ee.png)



## Many available datasets



Let's [take a look&#x2026;](https://developers.google.com/earth-engine/datasets/)



## Code Editor



One way to access [Google Earth Engine](https://code.earthengine.google.com/)



## Let's install the GEE Python API



    conda install -c conda-forge google-api-python-client earthengine-api
    earthengine authenticate



## Let's test the installation



In [1]:
import ee
ee.Initialize()
image = ee.Image('srtm90_v4')
print(image.getInfo())

## Now let's look at a map



In [1]:
from IPython.display import Image
Image(url=srtm.getThumbUrl({'min':0, 'max':3000}))

-   Unfortunately visualization is bit limited with Python
-   In some cases, the Code Editor might be better suited for our analysis



## By default, the image will have global extent so let's try to visualize a region



In [1]:
point = ee.Geometry.Point(-72.5199, 42.373)
region_amherst_area = point.buffer(500000).bounds().getInfo()['coordinates']
Image(url=srtm.getThumbUrl({'min':0, 'max':1000, 'region':region_amherst_area}))

## Working with image collections



Let's go to the [Earth Engine Public Data Catalog](https://earthengine.google.org/#index) and look for the Landsat-8 TOA reflectance data



## Create a reference to the image collection



In [1]:
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')

## Filter the collection for a year



In [1]:
filtered = l8.filterDate('2013-01-01', '2013-12-31');

## Mosaic the images and calculate the median



In [1]:
l8_image = filtered.median()

## Let's create a map



In [1]:
Image(url=l8_image.getThumbUrl({'region':region_amherst_area}))

## Let's get a true-color image



In [1]:
Image(url=l8_image.getThumbUrl({
    'region':region_amherst_area,
    'bands':'B4,B3,B2',
    'min':0,
    'max':0.3
}))

## Now let's get a false-color image



In [1]:
Image(url=l8_image.getThumbUrl({
    'region':region_amherst_area,
    'bands':'B5,B4,B3',
    'min':0,
    'max':0.3
}))

## Time series analysis with Landsat data



In [1]:
%matplotlib inline
import datetime
import numpy as np
from matplotlib import dates
import matplotlib.dates as mdates

Define the bands we'd like to plot and the time period



In [1]:
xBand = 'time'
yBandList = ['B1', 'B2', 'B3', 'B4', 'B5']
startTime = datetime.datetime(2014, 1, 1)
endTime = datetime.datetime(2017, 1, 1)

Construct a filtered image collection (let's pick Landsat-8) and extract information for a specific point



In [1]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(startTime, endTime)
point = {'type':'Point', 'coordinates':[ -116.88629, 36.56122]};  # death valley (should be stable)
info = collection.getRegion(point,500).getInfo()

## What is contained in info?



In [1]:
header = info[0]
header

In [1]:
data = np.array(info[1:])
data.shape

## Get the time information



In [1]:
it = header.index('time')
time = np.array([datetime.datetime.fromtimestamp(i/1000) for i in (data[:, it].astype(float))])
time

## Extract the data (bands) we want



In [1]:
iBands = [header.index(b) for b in yBandList]
yData = data[0:,iBands].astype(np.float)

## Plot Bands 3 and 4



In [1]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(111)
ax1.plot(time, yData[:,2], 'o', color="red", label="Band 3")
ax1.plot(time, yData[:,3], 'o', color="magenta",  label="Band 4")
ax1.legend(loc='best')
ax1.grid(True)
ax1.set_ylabel('Band Values')

NDVI is given by $$\frac{NIR - RED}{NIR + RED}$$

Let's calculate it



In [1]:
band3 = yData[:, 2]
band5 = yData[:, 4]
ndvi = (band5 - band3) / (band5 + band3)

In [1]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(111)
i = np.argsort(time)
ax1.plot(time[i], ndvi[i], '-o', color="black", label="NDVI")
ax1.grid(True)
ax1.set_ylabel('NDVI')

## What if we want to calculate NDVI images?



We will write a function that calculated NDVI



In [1]:
def AddBandNDVI(img):
    red = img.select('B3')
    nir = img.select('B5')
    ndvi = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')
    return img.addBands(ndvi)

In [1]:
ndvi = collection.map(AddBandNDVI)
print(ndvi.select('ndvi').getRegion(point, 500).getInfo())

## Did someone say interactive maps?



In [1]:
import ipyleaflet
import ipywidgets
import IPython.display

Define a colormap for NDVI



In [1]:
sld_ndvi = '''
<RasterSymbolizer>\
  <ChannelSelection>\
    <GrayChannel>\
      <SourceChannelName>ndvi</SourceChannelName>\
    </GrayChannel>\
  </ChannelSelection>\
  <ColorMap>\
    <ColorMapEntry color="#000000" quantity="0.0" />\
    <ColorMapEntry color="#BFBD27" quantity="0.1" />\
    <ColorMapEntry color="#1EBFBE" quantity="0.3" />\
    <ColorMapEntry color="#29FD2F" quantity="0.5" />\
    <ColorMapEntry color="#1CBD20" quantity="0.7" />\
    <ColorMapEntry color="#0F7E12" quantity="0.9" />\
  </ColorMap>\
</RasterSymbolizer>
'''
ndvi_legend = ipywidgets.HTML('''
<form>
 <fieldset>
  <legend>NDVI:</legend>
  <pre style="text-align:center;background-color:#777777;color:white">0.0</pre>
  <pre style="text-align:center;background-color:#BFBD27">0.1</pre>
  <pre style="text-align:center;background-color:#1EBFBE">0.3</pre>
  <pre style="text-align:center;background-color:#29FD2F">0.5</pre>
  <pre style="text-align:center;background-color:#1CBD20">0.7</pre>
  <pre style="text-align:center;background-color:#0F7E12">0.9</pre>
 </fieldset>
</form>
''',
layout=ipywidgets.Layout(width='100px'))

## and then define a helper function



In [1]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

## Set up our interactive map



In [1]:
map_ndvi = ipyleaflet.Map(
    zoom=4,
    center=(-72.5199, 42.373),
    layout={'height':'400px', 'width': '800px'}
)
display(ipywidgets.HBox([map_ndvi, ndvi_legend]))

## and add the NDVI layer



In [1]:
map_ndvi.add_layer(
    ipyleaflet.TileLayer(
        url=GetTileLayerUrl(
            ndvi.filterDate('2014-01-01', '2014-03-01')
                    .max()
                    .sldStyle(sld_ndvi)
        )
    )
)

In [1]:
map_ndvi